# Master TIDE - Conférences Python 2020

Francis Wolinski

&copy; 2020 Yotta Conseil

**Web Scraping avec requests et beautifulsoup**

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [ ]:
# imports
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

from IPython.display import IFrame

In [ ]:
# 1er essai avec leboncoin

r = requests.get('https://www.leboncoin.fr/annonces/offres/ile_de_france/')
r

Voir les codes des réponses du protocole HTTP: https://developer.mozilla.org/fr/docs/Web/HTTP/Status

In [ ]:
# contenu
r.content

Nécessite le script Python `server.py` lancé dans un shell avec la commande :
<code>python server.py --bind 127.0.0.1</code> sur Windows ou <code>python server.py --bind 0.0.0.0</code> sur MacOS.

In [ ]:
# avec run de server.py
r = requests.get('http://127.0.0.1:8000')  # http://0.0.0.0:8000
r

In [ ]:
# avec run de server.py
r.content

In [ ]:
# avec run de server.py
IFrame('http://127.0.0.1:8000', width=800, height=200)  # http://0.0.0.0:8000

In [ ]:
# headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate',
}

In [ ]:
# 2nd essai avec leboncoin
r = requests.get('https://www.leboncoin.fr/annonces/offres/ile_de_france/', headers=headers)
r

In [ ]:
# BeautifulSoup
soup = BeautifulSoup(r.content)
soup

In [ ]:
# essai avec craigslist
r = requests.get('https://paris.craigslist.org/d/locations-de-vacances/search/vac')
r

In [ ]:
# BeautifulSoup
soup = BeautifulSoup(r.content)
soup

BeautifulSoup possède 2 méthodes intéressantes :
    
- `find(tag, attrs)` : trouve le premier tag avec les attributs `attrs`
- `findAll(tag, attrs)` : trouve tous les tags avec les attributs `attrs`

In [ ]:
# exploration du HTML
# tag li avec class="result-row"

li_tag = soup.find('li', attrs={'class': 'result-row'})
print(li_tag)

La technique consiste par exemple à alimenter une liste de dictionnaires avec les valeurs trouvées pour chaque item et ensuite à le transformer en `DataFrame` :
- soit en utilisant `tag.attrs['attr']` pour collecter la valeur `attr` du tag `<tag attr=value>`
- soit en utilisant `tag.text` pour collecter la valeur `<tag>text</tag>`
- éventuellement en recherchant dans un nouveau tag à l'intérieur d'un tag donné

In [ ]:
# collecte des informations
# "time"
# "data-id"
# "titre"
# "price"
# "housing"
# "hood"
# "data-ids" (images)

rows = []

for li_tag in soup.findAll('li', attrs={'class': 'result-row'}):
    row = {}
    t = li_tag.find('time')
    row['datetime'] = t.attrs['datetime']
    # à compléter
    rows.append(row)
    
df = pd.DataFrame(rows)
df

In [ ]:
# collecte des photos
# traitement des "data-ids"
# séparation des formats et des noms de fichier
# from javascript
imageConfig = {"1":{"hostname":"https://images.craigslist.org","sizes":["50x50c","300x300","600x450","1200x900"]},
               "0":{"hostname":"https://images.craigslist.org","sizes":["50x50c","300x300","600x450"]},
               "2":{"hostname":"https://images.craigslist.org","sizes":["50x50c","300x300","600x450","1200x900"]}}

In [ ]:
# récupération des photos
from urllib import request
from shutil import copyfileobj

# data-ids
img = 'xxxx'
size = '50x50c'
filename = '{}_{}.jpg'.format(img, size)
url = 'https://images.craigslist.org/{}'.format(filename)

# get the file from the web and save it locally
with request.urlopen(url) as response, open(filename, 'wb') as out_file:
    copyfileobj(response, out_file)